Finally, we write the code file itself.  Most of the text below will appear literally in the file.  Statements in single curly braces will be substituted by the `format` statement at the end of the string; double curly braces will just appear as literal braces in the output.

In the following, the ODE system directly integrates the quaternion rotors describing the spin directions and orbital angular velocity vector.  This allows us to reduce the number of variables in the system and automatically satisfy certain constraints, which should lead to better numerical behavior.

Note that in this formulation, `Phi` is a completely unnecessary variable.  It doesn't need to be computed because it is implict in the definition of the frame.  It is calculated here simply because it is easy to do, and might be handy to have around.  Of course, it could be derived from knowledge of the frame alone.

In [5]:
# Always run this first
# NOTE: Do not define new basic variables in this notebook;
#       define them in Variables_Q.ipynb.  Use this notebook
#       to define new expressions built from those variables.

from __future__ import division # This needs to be here, even though it's in Variables.py
import sys
sys.path.insert(0, '..') # Look for modules in directory above this one
execfile('../Utilities/ExecNotebook.ipy')
UseQuaternions = True
from sympy import N
from sympy import Rational as frac # Rename for similarity to latex
execnotebook('../PNTerms/Variables_Q.ipynb')
from Utilities import CodeOutput
execnotebook('../PNTerms/WaveformModes.ipynb')

from __future__ import division # This needs to be here, even though it's in Variables.py
import sys
sys.path.insert(0, '..') # Look for modules in directory above this one
execfile('../Utilities/ExecNotebook.ipy')
from sympy import N
from sympy import Rational as frac # Rename for similarity to latex
execnotebook('../PNTerms/Variables_Q.ipynb')
from Utilities import CodeOutput
execnotebook('../PNTerms/WaveformModes.ipynb')

WaveformModeTerms = [WaveformModes_NoSpin, WaveformModes_Spin_Symm, WaveformModes_Spin_Asymm]
for Term in WaveformModeTerms:
    PNVariables.update(Term)
    
# For some reason I have to both overwrite these variables and pop them...
PNVariables.AddVariable('nHat', constant=True, fundamental=True,
                        substitution_atoms=[], substitution='Quaternions::xHat', datatype='Quaternions::Quaternion');
PNVariables.AddVariable('lambdaHat', constant=True, fundamental=True,
                        substitution_atoms=[], substitution='Quaternions::yHat', datatype='Quaternions::Quaternion');
PNVariables.AddVariable('ellHat', constant=True, fundamental=True,
                        substitution_atoms=[], substitution='Quaternions::zHat', datatype='Quaternions::Quaternion');
PNVariables.pop(nHat);
PNVariables.pop(lambdaHat);
PNVariables.pop(ellHat);

PNVariables.AddBasicVariables('chiVec1,chiVec2', datatype='Quaternions::Quaternion')

PNVariables.AddDerivedVariable('chi1_n', substitution_atoms=[chiVec1,nHat], substitution='chiVec1[1]');
PNVariables.AddDerivedVariable('chi1_lambda', substitution_atoms=[chiVec1,lambdaHat], substitution='chiVec1[2]');
PNVariables.AddDerivedVariable('chi1_ell', substitution_atoms=[chiVec1,ellHat], substitution='chiVec1[3]');
PNVariables.AddDerivedVariable('chi2_n', substitution_atoms=[chiVec2,nHat], substitution='chiVec2[1]');
PNVariables.AddDerivedVariable('chi2_lambda', substitution_atoms=[chiVec2,lambdaHat], substitution='chiVec2[2]');
PNVariables.AddDerivedVariable('chi2_ell', substitution_atoms=[chiVec2,ellHat], substitution='chiVec2[3]');

"\nfrom __future__ import division # This needs to be here, even though it's in Variables.py\nimport sys\nsys.path.insert(0, '..') # Look for modules in directory above this one\nexecfile('../Utilities/ExecNotebook.ipy')\nfrom sympy import N\nfrom sympy import Rational as frac # Rename for similarity to latex\nexecnotebook('../PNTerms/Variables_Q.ipynb')\nfrom Utilities import CodeOutput\nexecnotebook('../PNTerms/WaveformModes.ipynb')\n\nWaveformModeTerms = [WaveformModes_NoSpin, WaveformModes_Spin_Symm, WaveformModes_Spin_Asymm]\nfor Term in WaveformModeTerms:\n    PNVariables.update(Term)\n    \n# For some reason I have to both overwrite these variables and pop them...\nPNVariables.AddVariable('nHat', constant=True, fundamental=True,\n                        substitution_atoms=[], substitution='Quaternions::xHat', datatype='Quaternions::Quaternion');\nPNVariables.AddVariable('lambdaHat', constant=True, fundamental=True,\n                        substitution_atoms=[], substitution='Qu

In [6]:
with open('PNApproximants_Q.py', 'w') as f :
    f.write("""# File produced automatically by OrbitalEvolutionCodeGen_Q.ipynb
from scipy.integrate import solve_ivp
import numpy as np
from numpy import conjugate, dot, exp, log, pi
from numpy import euler_gamma as EulerGamma
import Quaternions
""")

    for PNOrder in [frac(n,2) for n in range(0,8)]:
        print("Working on {0} PN...".format(PNOrder))
        PNOrderString = str(N(PNOrder,2)).replace('.','p')
        %cd -q ../PNTerms
        execnotebook('OrbitalEvolution.ipynb')
        ExpressionsForMemberFunctions = CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities)
        execnotebook('AngularMomentum.ipynb')
        %cd -q -
        PrecessionVelocities.AddDerivedVariable('OrbitalAngularMomentum',
            AngularMomentumExpression(PNOrder=PNOrder),
            datatype=ellHat.datatype)
        CodeConstructor.AddDependencies(PrecessionVelocities)
        print(CodeConstructor.Atoms)
        # Start the class, write the declarations, initializations, etc.
        f.write("""
class TaylorTn_{PNOrderString}PN_Q : 
    def TaylorTn_{PNOrderString}PN_Q({InputArguments}) :
{Initializations}
        Phi=0.0
        EvolveSpin1=dot(S_chi1,S_chi1)>1e-12
        EvolveSpin2=dot(S_chi2,S_chi2)>1e-12

        def Recalculate(t, y):
            v = y[0];
            rfrak_chi1_x = y[1];
            rfrak_chi1_y = y[2];
            rfrak_chi2_x = y[3];
            rfrak_chi2_y = y[4];
            rfrak_frame_x = y[5];
            rfrak_frame_y = y[6];
            rfrak_frame_z = y[7];
            Phi = y[8];
{Evaluations}

{MemberFunctions}
""".format(PNOrderString=PNOrderString,
           InputArguments=CodeConstructor.CppInputArguments(11),
           Declarations=CodeConstructor.CppDeclarations(2),
           Initializations=CodeConstructor.CppInitializations(4),
           Evaluations=CodeConstructor.CppEvaluations(4),
           MemberFunctions=CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities).CppExpressionsAsFunctions(2)))

        # Write the external interfaces to the ODE stepper
        for n,Expressions in zip([1], [T1Expressions]):
            f.write("""
        #Evolve PN
        def TaylorT{n}(t, y):
            Recalculate(t, y)
            if v>=1.0: 
                return GSL_EDOM # Beyond domain of PN validity
{Computations}
            if dvdt_T{n}<1.0e-12:
                return GSL_EDIVERGE # v is decreasing
            return CommonRHS(dvdt_T{n}, y)
""".format(n=n,
           Computations=CodeOutput.CodeConstructor(PNVariables, Expressions).CppEvaluateExpressions() ))

        # Finish up, writing the common RHS function and closing the class
        f.write("""
        def CommonRHS(dvdt, y):
            dydt=np.zeros(9)
            rfrak_frame=np.zeros(3)
            rfrak_frame[0] = y[5]
            rfrak_frame[1] = y[6]
            rfrak_frame[2] = y[7]
            rfrakdot_frame = Quaternions.FrameFromAngularVelocity_Integrand(rfrak_frame, OmegaVec().vec())
            dydt[0] = dvdt
            if(EvolveSpin1):
                Quaternions.FrameFromAngularVelocity_2D_Integrand(y[1], y[2],\\
                    (S_chi1.inverse()*OmegaVec_chiVec_1()*S_chi1).vec(),dydt[1], dydt[2])
            else:
                dydt[1] = 0.0
                dydt[2] = 0.0
            if(EvolveSpin2):
                Quaternions.FrameFromAngularVelocity_2D_Integrand(y[3], y[4],\\
                    (S_chi2.inverse()*OmegaVec_chiVec_2()*S_chi2).vec(),dydt[3], dydt[4])
            else:
                dydt[3] = 0.0
                dydt[4] = 0.0
            dydt[5] = rfrakdot_frame[0]
            dydt[6] = rfrakdot_frame[1]
            dydt[7] = rfrakdot_frame[2]
            dydt[8] = v*v*v/M

            return dydt
        
        
        y=solve_ivp(TaylorT{n}, [0,20000], [0.01**(1/3),rfrak_chi1_x,\\
            rfrak_chi1_y,rfrak_chi2_x,rfrak_chi2_y,rfrak_frame_x,\\
            rfrak_frame_y,rfrak_frame_z,Phi], method='DOP853',\\
            t_eval=np.arange(0,20000,100), dense_output=True)
            

#        def TaylorT{n}_phi(t,phi):
#            dphidt_T{n}=v_of_t.y[0][v_of_t.t>=t][0]**3.0/M
#            return dphidt_T{n}
#        phi_of_t=solve_ivp(TaylorT{n}_phi, [0, v_of_t.t[-1]], [0.0], method='RK45', t_eval=v_of_t.t)
        
        #Calculate waveform modes
""".format(n=n,PNOrderString=PNOrderString))


    # Calculate waveform modes
        print("Working on {0} PN...".format(PNOrder))
        PNOrderString = str(N(PNOrder,2)).replace('.','p')
        ModeExpressions = PNCollection()
        from textwrap import TextWrapper
        wrapper = TextWrapper(width=120)
        wrapper.initial_indent = ' '*4
        wrapper.subsequent_indent = wrapper.initial_indent+'  '
        LM = [[ell,m] for ell in range(2,ellMax+1) for m in range(-ell,ell+1)]
        Evaluations = [wrapper.fill('std::vector<std::complex<double> > Modes({0});'.format(len(LM))),
                       wrapper.fill('std::complex<double> Symm, Asymm;')]
        for ell in range(2, ellMax+1):
            for m in range(0, ell+1):
                Symm = (SymmetricWaveformModes([ell,m],PNOrder=PNOrder)).subs(log(v), logv)
                Asymm = (AsymmetricWaveformModes([ell,m],PNOrder=PNOrder)).subs(log(v), logv)
                code1 = "Symm = {0};".format(sympy.ccode(Symm))
                code2 = "Asymm = {0};".format(sympy.ccode(Asymm))
                code3 = "Modes[{2}] = Symm + Asymm;".format(ell, m, LM.index([ell,m]))
                code4 = "Modes[{2}] = {3}std::conj(Symm - Asymm);".format(ell, -m, LM.index([ell,-m]),
                                                                               ('' if ((ell%2)==0) else '-'))
                ModeExpressions.AddDerivedVariable('rhOverM_{0}_{1}_Symm'.format(ell,m),
                                                   Symm, datatype='std::complex<double>')
                ModeExpressions.AddDerivedVariable('rhOverM_{0}_{1}_Asymm'.format(ell,m),
                                                   Asymm, datatype='std::complex<double>')
                Evaluations.append(wrapper.fill("// (ell, m) = ({0}, +/- {1})".format(ell, m)))
                Evaluations.append(wrapper.fill(code1))
                Evaluations.append(wrapper.fill(code2))
                Evaluations.append(wrapper.fill(code3))
                if m!=0:
                    Evaluations.append(wrapper.fill(code4))
        CodeConstructor = CodeOutput.CodeConstructor(PNVariables, ModeExpressions)
        if(PNOrder>0.9):
            Updates = """    chiVec1 = Quaternions::Quaternion(0., chi1_n_k, chi1_lambda_k, chi1_ell_k);
    chiVec2 = Quaternions::Quaternion(0., chi2_n_k, chi2_lambda_k, chi2_ell_k);
"""
        else:
            Updates = ""
        # Start the class, write the declarations, initializations, etc.
        f.write("""

class WaveformModes_{PNOrderString}PN : public WaveformModes_Base {{
private:
{Declarations}

public:
  WaveformModes_{PNOrderString}PN({InputArguments}) :
{Initializations}
  {{ }}

  using WaveformModes_Base::operator();

  std::vector<std::complex<double> > operator()(
    const double v_k,
    const double chi1_n_k, const double chi1_lambda_k, const double chi1_ell_k,
    const double chi2_n_k, const double chi2_lambda_k, const double chi2_ell_k)
  {{
    v = v_k;
{Updates}
{Evaluations}

{Computations}

    return Modes;
  }}

}}; // class WaveformModes_{PNOrderString}PN : public WaveformModes_Base
""".format(PNOrderString=PNOrderString,
           InputArguments=CodeConstructor.CppInputArguments(22),
           Declarations=CodeConstructor.CppDeclarations(2),
           Initializations=CodeConstructor.CppInitializations(4),
           Updates=Updates,
           Evaluations=CodeConstructor.CppEvaluations(4),
           MemberFunctions=CodeConstructor.CppExpressionsAsFunctions(2),
           Computations='\n'.join(Evaluations) ))

print("All done")

Working on 0 PN...
[xHat, yHat, zHat, M1, M2, v, S_chi1, S_chi2, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y, rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, M, delta, nu, R, nHat, ellHat, R_S1, R_S2, chiVec1, chiVec2, chi1chi1, chi1chi2, chi2chi2, chi1_n, chi1_ell, chi2_n, chi2_ell, S_ell, S_n, Sigma_ell, Sigma_n, chi_s_ell, chi_a_ell, logv, Fcal_coeff, Fcal_0, Fcal_2, Fcal_3, Fcal_4, Fcal_5, Fcal_6, Fcal_lnv_6, Fcal_7, Fcal_SQ_4, Fcal_SO_3, Fcal_SO_5, Fcal_SO_6, Fcal_SO_7, E_0, E_2, E_4, E_6, E_SQ_4, E_SO_3, E_SO_5, E_SO_7]
Working on 0 PN...
Working on 1/2 PN...
[xHat, yHat, zHat, M1, M2, v, S_chi1, S_chi2, rfrak_chi1_x, rfrak_chi1_y, rfrak_chi2_x, rfrak_chi2_y, rfrak_frame_x, rfrak_frame_y, rfrak_frame_z, M, delta, nu, R, nHat, ellHat, R_S1, R_S2, chiVec1, chiVec2, chi1chi1, chi1chi2, chi2chi2, chi1_n, chi1_ell, chi2_n, chi2_ell, S_ell, S_n, Sigma_ell, Sigma_n, chi_s_ell, chi_a_ell, logv, Fcal_coeff, Fcal_0, Fcal_2, Fcal_3, Fcal_4, Fcal_5, Fcal_6, Fcal_lnv_6, Fcal_7, Fcal_S